In [5]:
import pandas as pd
df = pd.read_csv('data/1003.csv')
df.head()

,timestamp,room_id,co2,temperature,humidity
0,2022-11-16 15:30:02+00:00,37239,1371,19.6,48
1,2022-11-16 15:32:02+00:00,37239,1367,19.6,48
2,2022-11-16 15:34:02+00:00,37239,1390,19.6,48
3,2022-11-16 15:36:02+00:00,37239,1384,19.6,48
4,2022-11-16 15:38:02+00:00,37239,1377,19.6,48
